# Scrape LinkedIn Using Selenium, Request and Beautiful Soup in Python

We are going to scrape Linkedin Jobs. More specifically, the following details will be scraped:
- Job Id
- Job title
- Seniority Level
- Location
- Job description
- number of candidats
- posted time ago

1. To scrape Job Ids, we will use `selenium` to navigate to this URL: `https://www.linkedin.com/jobs/search?`.

`chromedriver` executable and your LinkedIn credentials are required here.

3. As explained [here](https://www.scrapingdog.com/blog/scrape-linkedin-jobs/), to scrape other details (level, description...), we will use a simple GET request (leveraging the `requests` library) to this URL: `https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/xxxxxx` where xxxxxx is the job ID.\
This is easier than using clicks from `selenium`.

Note that I tried to scrape Job IDs using the guest URL `https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?` but the results were imprecise.

# I-Scraping Linkedin Jobs IDs using selenium and BeautifulSoup

In [12]:
# pip install selenium 
# pip install beautifulsoup4

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
import requests

import time, datetime
import pandas as pd
import numpy as np
import math, re, sys
import warnings
warnings.filterwarnings("ignore")

## Login to Linkedin using selenium

🔑 **Note:**

1. To use `selenium`, we need a web driver. For instance, the `chromedriver` can be downloaded from [here](https://chromedriver.chromium.org/downloads).\
   Next, we add the chromedriver to the project directory.

3. Linkedin credentials (email address and password) are also required. You can save them here: `../data/user_credentials.txt`

In [13]:
# Get User Credentials
with open('/Users/Jason/Desktop/is3107/is3107_DataEngineering/user_credentials.txt', 'r',encoding="utf-8") as file:
    user_credentials = file.readlines()
    user_credentials = [line.rstrip() for line in user_credentials]
    
my_email,my_pwd = user_credentials[0],user_credentials[1]
my_email,my_pwd

('bt4222project@gmail.com', 'bt4222project')

In [14]:
# 1. Instanciate the chrome service
chromedriver_path = "C:/Users/Jason/Desktop/IS3107/chromedriver.exe"
service = Service(executable_path=chromedriver_path)

# 2. Instanciate the webdriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options, service=service)

# 3. Open the LinkedIn login page
driver.get('https://www.linkedin.com/login')
time.sleep(5) # waiting for the page to load

# 4. Enter email address & password
email_input = driver.find_element(By.ID, 'username')
password_input = driver.find_element(By.ID, 'password')
email_input.send_keys(my_email)
password_input.send_keys(my_pwd)

# 5. Click the login button
password_input.send_keys(Keys.ENTER)

time.sleep(10)

We will be logged into LinkedIn after running the above code.

## Scraping Linkedin Jobs IDs

1. Set the search query parameters: keywords (ie. Job title) and location;
2. Search results are displayed on many pages: `25` jobs are listed on each page;
3. We will navigate to every page using the `start` parameter (0,25,50...);
4. We need to scroll to the bottom of the page to load the full data;
5. To get Job Ids, we will parse the HTML content of the page using BeautifulSoup.

In [15]:
List_Job_IDs = []

In [16]:
# Create a function 'Scroll to the bottom'. 

# time.sleep() function is used to provide extra time for the webpage to load. 
# I used 120 seconds. If the 25 jobs have not loaded during this period, we can make adjust it and test again.

def scroll_to_bottom(driver,sleep_time=120):
    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            break
        last_height = new_height
    
    time.sleep(sleep_time)  

In [17]:
# Navigate to the first page (start=0) and scroll to the bottom of the page

keywords = 'machine%20learning%20engineer'
location = 'singapore'
start = 0

url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={start}'
url = requests.utils.requote_uri(url)
driver.get(url)
scroll_to_bottom(driver,sleep_time=120)

In [18]:
# Get number of jobs found and number of pages:

# Parse the HTML content of the page using BeautifulSoup.
soup = BeautifulSoup(driver.page_source, 'html.parser')

try:
    div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
    number_of_jobs = int(div_number_of_jobs.find('span').get_text().strip().split()[0])
except:
    number_of_jobs = 0
    
number_of_pages=math.ceil(number_of_jobs/25)
print("number_of_jobs:",number_of_jobs)
print("number_of_pages:",number_of_pages)

number_of_jobs: 388
number_of_pages: 16


In [19]:
# Get Job Ids present on the first page.

def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    
    job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
    for job_posting in job_postings:
        Job_ID = job_posting.get('data-occludable-job-id')
        Job_Ids_on_the_page.append(Job_ID)
        # job_title = job_posting.find('a', class_='job-card-list__title').get_text().strip()
        # location = job_posting.find('li', class_='job-card-container__metadata-item').get_text().strip()
    
    return Job_Ids_on_the_page    

Jobs_on_this_page = find_Job_Ids(soup)
List_Job_IDs.extend(Jobs_on_this_page)

Now that we've scraped the job IDs and number of results from the first page, let's iterate over the remaining pages.

### Iterate over the remaining pages

In [20]:
if number_of_pages>1:
    
    for page_num in range(1,number_of_pages):
        print(f"Scraping page: {page_num}",end="...")
        
        # Navigate to page
        url = f'https://www.linkedin.com/jobs/search/?keywords={keywords}&location={location}&start={25 * page_num}'
        url = requests.utils.requote_uri(url)
        driver.get(url)
        scroll_to_bottom(driver)

        # Parse the HTML content of the page using BeautifulSoup.
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Get Job Ids present on the page.
        Jobs_on_this_page = find_Job_Ids(soup)
        List_Job_IDs.extend(Jobs_on_this_page)  
        print(f'Jobs found:{len(Jobs_on_this_page)}')

pd.DataFrame({"Job_Id":List_Job_IDs}).to_csv('Job_Ids.csv',index=False)

Scraping page: 1...Jobs found:25
Scraping page: 2...Jobs found:25
Scraping page: 3...Jobs found:25
Scraping page: 4...Jobs found:25
Scraping page: 5...Jobs found:25
Scraping page: 6...Jobs found:25
Scraping page: 7...Jobs found:25
Scraping page: 8...Jobs found:25
Scraping page: 9...Jobs found:25
Scraping page: 10...Jobs found:25
Scraping page: 11...Jobs found:25
Scraping page: 12...Jobs found:25
Scraping page: 13...Jobs found:25
Scraping page: 14...Jobs found:25
Scraping page: 15...Jobs found:25


In [21]:
## Close the browser and shut down the ChromiumDriver executable that
# is started when starting the ChromiumDriver. 
driver.quit()

## Scraping Job description using requests and BeautifulSoup
https://www.scrapingdog.com/blog/scrape-linkedin-jobs/

In [22]:
import requests
from bs4 import BeautifulSoup

list_job_IDs = pd.read_csv("Job_Ids.csv").Job_Id.to_list()

In [23]:
list_job_IDs = list_job_IDs
list_job_IDs

[3903271990,
 3563372269,
 3742413272,
 3629531573,
 3904603735,
 3642805607,
 3727667827,
 3817555266,
 3870752167,
 3642802633,
 3902332409,
 3903662851,
 3864740884,
 3902336096,
 3802032623,
 3895188763,
 3854396993,
 3872735968,
 3477666274,
 3879302536,
 3876044260,
 3854396970,
 3889132367,
 3902421650,
 3832648465,
 3691338351,
 3592057758,
 3776577476,
 3685765704,
 3839793558,
 3839796369,
 3876047030,
 3901741161,
 3798299351,
 3834843637,
 3905800020,
 3853302641,
 3774414029,
 3900671378,
 3888585441,
 3902333320,
 3797480400,
 3888900324,
 3843915160,
 3860132771,
 3894036088,
 3456898471,
 3861717399,
 3906403105,
 3901733882,
 3615401816,
 3691695040,
 3576765601,
 3641402868,
 3894458926,
 3787667914,
 3654449884,
 3848140577,
 3878971862,
 3654451566,
 3894033429,
 3908158459,
 3768185071,
 3881468106,
 3876698428,
 3861930853,
 3854380245,
 3891264165,
 3849976187,
 3837264781,
 3907007496,
 3899982688,
 3890802396,
 3888062856,
 3894699108,
 3678261330,
 3675017494,

In [24]:
def remove_tags(html):
    '''remove html tags from BeautifulSoup.text'''
 
    # parse html content
    soup = BeautifulSoup(html, "html.parser")
 
    for data in soup(['style', 'script']):
        # Remove tags
        data.decompose()
 
    # return data by retrieving the tag content
    return ' '.join(soup.stripped_strings)

In [25]:
job_url='https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{}'
job={}
list_jobs=[]

for j in range(0,len(list_job_IDs)):
    print(f"{j+1} ... read jobId:{list_job_IDs[j]}")

    resp = requests.get(job_url.format(list_job_IDs[j]))
    soup=BeautifulSoup(resp.text,'html.parser')
    # print(soup.prettify()) 

    job["Job_ID"] = list_job_IDs[j] 
    # try:
    #     job["Job_html"] = resp.content
    # except:
    #     job["Job_html"]=None

    try: # remove tags
        job["Job_txt"] = remove_tags(resp.content)
    except:
        job["Job_txt"] = None
    
    try:
        job["company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
    except:
        job["company"]=None

    try:
        job["job-title"]=soup.find("div",{"class":"top-card-layout__entity-info"}).find("a").text.strip()
    except:
        job["job-title"]=None

    try:
        job["level"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
    except:
        job["level"]=None

    try:
        job["location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
    except:
        job["location"]=None

    try:
        job["posted-time-ago"]=soup.find("span",{"class":"posted-time-ago__text topcard__flavor--metadata"}).text.strip()
    except:
        job["posted-time-ago"]=None

    try:
        nb_candidats = soup.find("span",{"class":"num-applicants__caption topcard__flavor--metadata topcard__flavor--bullet"}).text.strip()
        nb_candidats = int(nb_candidats.split()[0])
        job["nb_candidats"]= nb_candidats
    except:
        job["nb_candidats"]=None

    list_jobs.append(job)
    job={}

# create a pandas Datadrame
jobs_DF = pd.DataFrame(list_jobs)

1 ... read jobId:3903271990
2 ... read jobId:3563372269
3 ... read jobId:3742413272
4 ... read jobId:3629531573
5 ... read jobId:3904603735
6 ... read jobId:3642805607
7 ... read jobId:3727667827
8 ... read jobId:3817555266
9 ... read jobId:3870752167
10 ... read jobId:3642802633
11 ... read jobId:3902332409
12 ... read jobId:3903662851
13 ... read jobId:3864740884
14 ... read jobId:3902336096
15 ... read jobId:3802032623
16 ... read jobId:3895188763
17 ... read jobId:3854396993
18 ... read jobId:3872735968
19 ... read jobId:3477666274
20 ... read jobId:3879302536
21 ... read jobId:3876044260
22 ... read jobId:3854396970
23 ... read jobId:3889132367
24 ... read jobId:3902421650
25 ... read jobId:3832648465
26 ... read jobId:3691338351
27 ... read jobId:3592057758
28 ... read jobId:3776577476
29 ... read jobId:3685765704
30 ... read jobId:3839793558
31 ... read jobId:3839796369
32 ... read jobId:3876047030
33 ... read jobId:3901741161
34 ... read jobId:3798299351
35 ... read jobId:38348

In [26]:
jobs_DF.head()

,Job_ID,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats
0,3903271990,Mobile Game Designer (System Design / Worldbui...,None,Mobile Game Designer (System Design / Worldbui...,Entry level,"Singapore, Singapore",2 days ago,34.0
1,3563372269,"Machine Learning Engineer, Business Integrity ...",TikTok,"Machine Learning Engineer, Business Integrity",Not Applicable,Singapore,1 month ago,90.0
2,3742413272,"Machine Learning Engineer, Smart Code TikTok S...",TikTok,"Machine Learning Engineer, Smart Code",Not Applicable,Singapore,1 month ago,72.0
3,3629531573,Senior /Machine Learning Engineer - DSC/EZ ST ...,ST Engineering,Senior /Machine Learning Engineer - DSC/EZ,Mid-Senior level,"Singapore, Singapore",3 weeks ago,NaN
4,3904603735,"Senior Backend Engineer Shopee Singapore, Sing...",Shopee,Senior Backend Engineer,Mid-Senior level,"Singapore, Singapore",None,NaN


🔑 **Note:**

Now we have scraped all Linkedin Job details. 

The next step is to process the data:
1. Create a posted_date column using posted_time_ago;
2. Clean up 'Job_description' (remove sentences like "Remove photo First name Last name Email Password ( 8 + characters )");
3. Clean up the 'level' column.

## Process data

In [27]:
def clean_Job_description(text):
    senetences_to_remove = ["Remove photo First name Last name Email Password (8+ characters) ",
                            "By clicking Agree & Join",
                            "you agree to the LinkedIn User Agreement",
                            "Privacy Policy and Cookie Policy",
                            "Continue Agree & Join or Apply on company website",
                            "Security verification",
                            "Close Already on LinkedIn ?",
                            "Close Already on LinkedIn?",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile , or create a new one",
                            "Sign in Save Save job Save this job with your existing LinkedIn profile, or create a new one",
                            "Your job seeking activity is only visible to you",
                            "Email Continue Welcome back"]
    for sentence in senetences_to_remove:
        result = text.find(sentence)
        if result>-1:
            text = text[:result] + text[result+len(sentence):] # remove sentence from text

    return text 

In [28]:
def get_posted_date(posted_time_ago,date_scraping):
    """Convert posted_time_ago to number of days.
    For example, 1 month ago is replaced by 30. 1 week by 7 and so on..."""
    posted_date = None
    
    try:
        details = posted_time_ago.split()
        N_DAYS_AGO = int(details[0])
        day_week_month_year = details[1] 
        if day_week_month_year.startswith("day"):
            N_DAYS_AGO = N_DAYS_AGO
        elif day_week_month_year.startswith("week"):
            N_DAYS_AGO = N_DAYS_AGO*7
        elif day_week_month_year.startswith("month"):
            N_DAYS_AGO = N_DAYS_AGO*30
        elif day_week_month_year.startswith("year"):
            N_DAYS_AGO = N_DAYS_AGO*365
        else:
            N_DAYS_AGO = None

        posted_date = date_scraping - datetime.timedelta(days=N_DAYS_AGO)
    except:
        posted_date = None

    return posted_date

In [29]:
jobs_DF['scraping_date'] = pd.to_datetime(datetime.date.today())
jobs_DF['posted_date'] = np.vectorize(get_posted_date)(jobs_DF['posted-time-ago'], jobs_DF['scraping_date'])

jobs_DF['Job_txt'] = jobs_DF['Job_txt'].apply(clean_Job_description)
jobs_DF.level = jobs_DF.level.apply(lambda x:x.replace("Employment type\n        \n\n          ","") if x is not None else x)

jobs_DF.head()

,Job_ID,Job_txt,company,job-title,level,location,posted-time-ago,nb_candidats,scraping_date,posted_date
0,3903271990,Mobile Game Designer (System Design / Worldbui...,None,Mobile Game Designer (System Design / Worldbui...,Entry level,"Singapore, Singapore",2 days ago,34.0,2024-04-23,2024-04-21
1,3563372269,"Machine Learning Engineer, Business Integrity ...",TikTok,"Machine Learning Engineer, Business Integrity",Not Applicable,Singapore,1 month ago,90.0,2024-04-23,2024-03-24
2,3742413272,"Machine Learning Engineer, Smart Code TikTok S...",TikTok,"Machine Learning Engineer, Smart Code",Not Applicable,Singapore,1 month ago,72.0,2024-04-23,2024-03-24
3,3629531573,Senior /Machine Learning Engineer - DSC/EZ ST ...,ST Engineering,Senior /Machine Learning Engineer - DSC/EZ,Mid-Senior level,"Singapore, Singapore",3 weeks ago,NaN,2024-04-23,2024-04-02
4,3904603735,"Senior Backend Engineer Shopee Singapore, Sing...",Shopee,Senior Backend Engineer,Mid-Senior level,"Singapore, Singapore",None,NaN,2024-04-23,NaT


## Save to json file

In [30]:
jobs_DF.to_json("machineLearningEngineer_scraped.json")
jobs_DF.to_csv("machineLearningEngineer_scraped.csv")